In [1]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "/Users/sohyun/Desktop/review data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "/Users/sohyun/Desktop/review data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "/Users/sohyun/Desktop/review data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [2]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [3]:
# Download the punkt tokenizer for sentence splitting
import nltk.data

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [5]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/sohyun/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/sohyun/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/Users/sohyun/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/sohyun/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/sohyun/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client 

In [6]:
# training and saving model

In [8]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-06-13 10:35:52,620 : INFO : 'pattern' package not found; tag filters are not available for English
2019-06-13 10:35:52,627 : INFO : collecting all words and their counts
2019-06-13 10:35:52,628 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-13 10:35:52,725 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-06-13 10:35:52,807 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2019-06-13 10:35:52,909 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2019-06-13 10:35:52,983 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-06-13 10:35:53,031 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-06-13 10:35:53,116 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-06-13 10:35:53,163 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-06-13 10:35:53,261 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-06-13 10:35:53,339 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-06-13 10:35:53,398 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-06-13 10:35:53,491 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

2019-06-13 10:35:58,478 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2019-06-13 10:35:58,573 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2019-06-13 10:35:58,666 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2019-06-13 10:35:58,762 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2019-06-13 10:35:58,860 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2019-06-13 10:35:58,908 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2019-06-13 10:35:58,908 : INFO : Loading a fresh vocabulary
2019-06-13 10:35:59,012 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2019-06-13 10:35:59,012 : INFO : effective_min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559

2019-06-13 10:36:52,812 : INFO : EPOCH 3 - PROGRESS: at 22.32% examples, 560262 words/s, in_qsize 8, out_qsize 0
2019-06-13 10:36:53,813 : INFO : EPOCH 3 - PROGRESS: at 27.25% examples, 571444 words/s, in_qsize 7, out_qsize 0
2019-06-13 10:36:54,832 : INFO : EPOCH 3 - PROGRESS: at 32.34% examples, 580038 words/s, in_qsize 7, out_qsize 0
2019-06-13 10:36:55,858 : INFO : EPOCH 3 - PROGRESS: at 36.29% examples, 568369 words/s, in_qsize 6, out_qsize 1
2019-06-13 10:36:56,875 : INFO : EPOCH 3 - PROGRESS: at 40.75% examples, 567773 words/s, in_qsize 8, out_qsize 1
2019-06-13 10:36:57,886 : INFO : EPOCH 3 - PROGRESS: at 43.31% examples, 543565 words/s, in_qsize 7, out_qsize 0
2019-06-13 10:36:58,887 : INFO : EPOCH 3 - PROGRESS: at 47.39% examples, 541532 words/s, in_qsize 7, out_qsize 0
2019-06-13 10:36:59,892 : INFO : EPOCH 3 - PROGRESS: at 51.08% examples, 535614 words/s, in_qsize 7, out_qsize 0
2019-06-13 10:37:00,903 : INFO : EPOCH 3 - PROGRESS: at 54.54% examples, 528114 words/s, in_qsiz

2019-06-13 10:37:55,922 : INFO : EPOCH - 5 : training on 17798082 raw words (12749930 effective words) took 20.0s, 636742 effective words/s
2019-06-13 10:37:55,922 : INFO : training on a 88990410 raw words (63748035 effective words) took 116.4s, 547543 effective words/s
2019-06-13 10:37:55,923 : INFO : precomputing L2-norms of word weight vectors
2019-06-13 10:37:55,957 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2019-06-13 10:37:55,959 : INFO : not storing attribute vectors_norm
2019-06-13 10:37:55,960 : INFO : not storing attribute cum_table
2019-06-13 10:37:56,530 : INFO : saved 300features_40minwords_10context


In [9]:
# exploring the model result

In [10]:
model.doesnt_match("man woman child kitchen".split())

/Users/sohyun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'kitchen'

In [11]:
model.most_similar("man")

/Users/sohyun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6318554282188416),
 ('lad', 0.5945581197738647),
 ('lady', 0.5849864482879639),
 ('guy', 0.5348923206329346),
 ('farmer', 0.5239083766937256),
 ('men', 0.5230334997177124),
 ('millionaire', 0.5216241478919983),
 ('monk', 0.5184000730514526),
 ('businessman', 0.512630820274353),
 ('person', 0.5095211863517761)]

In [12]:
model.most_similar("queen")

/Users/sohyun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('princess', 0.6407474279403687),
 ('maid', 0.6254844665527344),
 ('bride', 0.6186751127243042),
 ('belle', 0.6068633794784546),
 ('victoria', 0.5981305837631226),
 ('mistress', 0.5968143343925476),
 ('goddess', 0.5934066772460938),
 ('desmond', 0.5878401398658752),
 ('stepmother', 0.5834432244300842),
 ('dame', 0.5813528895378113)]

In [13]:
model.most_similar("awful")

/Users/sohyun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7712309956550598),
 ('atrocious', 0.7345525026321411),
 ('horrible', 0.7138197422027588),
 ('abysmal', 0.7053906917572021),
 ('dreadful', 0.7042431831359863),
 ('horrendous', 0.6868628859519958),
 ('appalling', 0.6740887761116028),
 ('horrid', 0.6633691191673279),
 ('lousy', 0.6420338153839111),
 ('amateurish', 0.6194838285446167)]

In [14]:
# tutorial 3

In [15]:
# load model

In [16]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

2019-06-13 10:45:13,750 : INFO : loading Word2Vec object from 300features_40minwords_10context
2019-06-13 10:45:14,158 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2019-06-13 10:45:14,159 : INFO : setting ignored attribute vectors_norm to None
2019-06-13 10:45:14,159 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2019-06-13 10:45:14,161 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2019-06-13 10:45:14,162 : INFO : setting ignored attribute cum_table to None
2019-06-13 10:45:14,165 : INFO : loaded 300features_40minwords_10context


In [19]:
model

In [20]:
# Attempt 1: Vector Averaging

In [28]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given paragraph
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    
    # Index2word is a list that contains the names of the words in the model's vocabulary. 
    # Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    
    # Loop over each word in the review and, if it is in the model's vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
     
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), 
    # calculate the average feature vector for each one and return a 2D numpy array 
    # Initialize a counter
    counter = 0.
     
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
     
    # Loop through the reviews
    for review in reviews:
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print("Review %d of %d" % (counter, len(reviews)))
    
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, \
           num_features)
       
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [22]:
# create average vectors for each paragraph

In [29]:
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000


/Users/sohyun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [30]:
# Attempt 2: k-means

In [31]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

from bs4 import BeautifulSoup
import re
import time

from nltk.corpus import stopwords
import nltk.data

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [33]:
model = Word2Vec.load("300features_40minwords_10context")
model

2019-06-13 11:18:19,276 : INFO : loading Word2Vec object from 300features_40minwords_10context
2019-06-13 11:18:19,696 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2019-06-13 11:18:19,697 : INFO : setting ignored attribute vectors_norm to None
2019-06-13 11:18:19,698 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2019-06-13 11:18:19,698 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2019-06-13 11:18:19,699 : INFO : setting ignored attribute cum_table to None
2019-06-13 11:18:19,699 : INFO : loaded 300features_40minwords_10context


In [34]:
type(model.wv.syn0)

/Users/sohyun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [35]:
model.wv.syn0.shape

/Users/sohyun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


(16490, 300)

In [36]:
# 단어 벡터에서 k-means를 실행하고 일부 클러스터를 찍어본다.
start = time.time() # 시작시간

# 클러스터의 크기 "k"를 어휘 크기의 1/5 이나 평균 5단어로 설정한다.
word_vectors = model.wv.syn0 # 어휘의 feature vector
num_clusters = word_vectors.shape[0] / 5
num_clusters = int(num_clusters)

# K means 를 정의하고 학습시킨다.
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# 끝난시간에서 시작시간을 빼서 걸린 시간을 구한다.
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

/Users/sohyun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """


Time taken for K Means clustering:  480.6833698749542 seconds.


In [37]:
# 각 어휘 단어를 클러스터 번호에 매핑되게 word/Index 사전을 만든다.
idx = list(idx)
names = model.wv.index2word
word_centroid_map = {names[i]: idx[i] for i in range(len(names))}
#     word_centroid_map = dict(zip( model.wv.index2word, idx ))

# 첫번째 클러스터의 처음 10개를 출력
for cluster in range(0,10):
    # 클러스터 번호를 출력
    print("\nCluster {}".format(cluster))
    
    # 클러스터번호와 클러스터에 있는 단어를 찍는다.
    words = []
    for i in range(0,len(list(word_centroid_map.values()))):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0
['leila']

Cluster 1
['roberts', 'stiles', 'raul', 'rigg']

Cluster 2
['door', 'wall', 'floor', 'window', 'tree', 'glass', 'toilet', 'bottle', 'wheel', 'sweat', 'ceiling', 'curtain', 'fridge']

Cluster 3
['inadvertently', 'unwittingly', 'creasy', 'superhuman']

Cluster 4
['spielberg', 'kubrick', 'cronenberg', 'bakshi']

Cluster 5
['discerning']

Cluster 6
['bright', 'colorful', 'lively', 'colourful', 'brightly']

Cluster 7
['joe', 'bill', 'mike', 'rob', 'larry', 'buck', 'mickey', 'dave', 'dante', 'clay', 'barney', 'moe', 'shemp']

Cluster 8
['theaters', 'cinemas', 'theatres']

Cluster 9
['meets', 'returns', 'arrives', 'encounters', 'travels', 'enters', 'visits', 'marries', 'owns', 'belonged', 'awakens', 'rents', 'nursing', 'attends', 'inherits', 'announcing']


In [38]:
# centroid 는 두 클러스터의 중심점을 정의 한 다음 중심점의 거리를 측정한 것
def create_bag_of_centroids( wordlist, word_centroid_map ):
    
    # 클러스터의 수는 word / centroid map에서 가장 높은 클러스트 인덱스와 같다.
    num_centroids = max( word_centroid_map.values() ) + 1
    
    # 속도를 위해 bag of centroids vector를 미리 할당한다.
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    
    # 루프를 돌며 단어가 word_centroid_map에 있다면
    # 해당되는 클러스터의 수를 하나씩 증가시켜 준다.
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    
    # bag of centroids를 반환한다.
    return bag_of_centroids

In [42]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [44]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "/Users/sohyun/Desktop/review data/BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
